In [ ]:
import pandas as pd
import os
import glob
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
import numpy as np

# Code to read in daily files and combine them into a single dataframe
path = './data'
all_files = glob.glob(os.path.join(path, "*.csv"))

dataframes = []
for file in all_files:
    location = os.path.basename(file).split('_')[0]
    df = pd.read_csv(file)
    df['Location'] = location
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

# Define unique locations and model types
locations = df['Location'].unique().tolist()
model_types = ['cmc', 'gfs', 'gefs', 'geps', 'cfs', 'ecmwf']

# Create dropdown for location selection
location_dropdown = widgets.Dropdown(
    options=locations,
    value=locations[0],
    description='Location:',
)

# Create checkboxes for model selection
model_checkboxes = [widgets.Checkbox(value=True, description=model_type) for model_type in model_types]
model_selection = widgets.VBox(model_checkboxes)

# Function to update plot based on user interaction
def update_plot(*args):
    selected_location = location_dropdown.value
    selected_models = [cb.description for cb in model_checkboxes if cb.value]

    # Filter data based on user selection
    data = df[df['Location'] == selected_location]
    
    # Clear current figure
    fig = go.Figure()

    # For each selected model, add trace to figure
    for model_type in selected_models:
        tmin_cols = [col for col in data.columns if 'TMin24' in col and model_type in col]
        tmax_cols = [col for col in data.columns if 'TMax24' in col and model_type in col]
        precip_cols = [col for col in data.columns if 'Precip24' in col and model_type in col]
        percentiles = np.linspace(0, 100, 11)
        
        # Plot Tmin24
        for i in range(len(percentiles)//2):
            p1 = np.nanpercentile(data[tmin_cols].values, percentiles[i], axis=1)
            p2 = np.nanpercentile(data[tmin_cols].values, percentiles[-(i+1)], axis=1)
            fig.add_trace(go.Scatter(x=data['Date'], y=p1, fill=None, mode='lines', line_color='blue', showlegend=False, hoverinfo='none'))
            fig.add_trace(go.Scatter(x=data['Date'], y=p2, fill='tonexty', mode='lines', line_color='blue', showlegend=False, hoverinfo='none'))

        # Plot Tmax24
        for i in range(len(percentiles)//2):
            p1 = np.nanpercentile(data[tmax_cols].values, percentiles[i], axis=1)
            p2 = np.nanpercentile(data[tmax_cols].values, percentiles[-(i+1)], axis=1)
            fig.add_trace(go.Scatter(x=data['Date'], y=p1, fill=None, mode='lines', line_color='red', showlegend=False, hoverinfo='none'))
            fig.add_trace(go.Scatter(x=data['Date'], y=p2, fill='tonexty', mode='lines', line_color='red', showlegend=False, hoverinfo='none'))

        # Plot Precip24
        for i in range(len(percentiles)//2):
            p1 = np.nanpercentile(data[precip_cols].values, percentiles[i], axis=1)
            p2 = np.nanpercentile(data[precip_cols].values, percentiles[-(i+1)], axis=1)
            fig.add_trace(go.Scatter(x=data['Date'], y=p1, fill=None, mode='lines', line_color='green', showlegend=False, hoverinfo='none'))
            fig.add_trace(go.Scatter(x=data['Date'], y=p2, fill='tonexty', mode='lines', line_color='green', showlegend=False, hoverinfo='none'))

    # Update plot layout to simplify less important information
    fig.update_layout(
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        plot_bgcolor='white'
    )
    
    # Update plot
    fig.show()

# Call update_plot whenever the selection changes
location_dropdown.observe(update_plot, 'value')
for cb in model_checkboxes:
    cb.observe(update_plot, 'value')

# Initialize plot
update_plot()

# Display widgets
display(location_dropdown)
display(model_selection)
